# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords','omw-1.4'])
from nltk.corpus import stopwords

import cProfile
import pandas as pd
import sqlite3
import re
import numpy as np

from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\panel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\panel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\panel\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\panel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\panel\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
# load data from database
# engine = create_engine('sqlite:///InsertDatabaseName.db')
cnxn = sqlite3.connect("messages.sqlite")
df = pd.read_sql("SELECT * FROM messages_classified",cnxn)
cnxn.close()
X = df.message
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [42]:
%reload_ext line_profiler

pattern_stopwords = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')

def tokenize(text):
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    text = pattern_stopwords.sub('',text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    
    for tok in word_tokenize(text):
        clean_tok = lemmatizer.lemmatize(tok)#.lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [37]:
sw_set = set(stopwords.words("english"))

In [46]:
# %lprun -f tokenize tokenize(X[0])
tokenize(X[0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

In [5]:
print(X.head(100).values)

['Weather update - a cold front from Cuba that could pass over Haiti'
 'Is the Hurricane over or is it not over'
 'Looking for someone but no name'
 'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.'
 'says: west side of Haiti, rest of the country today and tonight'
 'Information about the National Palace-' 'Storm at sacred heart of jesus'
 'Please, we need tents and water. We are in Silo, Thank you!'
 'I would like to receive the messages, thank you'
 'I am in Croix-des-Bouquets. We have health issues. They ( workers ) are in Santo 15. ( an area in Croix-des-Bouquets )'
 "There's nothing to eat and water, we starving and thirsty."
 'I am in Petionville. I need more information regarding 4636'
 'I am in Thomassin number 32, in the area named Pyron. I would like to have some water. Thank God we are fine, but we desperately need water. Thanks'
 "Let's do it together, need food in Delma 75, in didine area"
 'More information on the 4636 

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [44]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10)))
], verbose=True)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
Y[:5]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, Y[['request','offer','aid_related']])

pipeline.fit(X_train, y_train)

# y_pred = pipeline.predict(X_test)

[Pipeline] .............. (step 1 of 3) Processing vect, total=   4.3s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   5.4s


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000002A450D39B40>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10)))],
         verbose=True)

In [47]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)
Y_pred = pipeline.predict(X_test)

[Pipeline] .............. (step 1 of 3) Processing vect, total=   4.3s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=  35.3s


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
Y_pred[:10]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],


In [48]:
for ix,col in enumerate(Y_train.columns):
    y_true = Y_test[col]
    y_pred = Y_pred[:,ix]
    target_names = ['0', '1']
    print(f"====={col}================")
    print(classification_report(y_true, y_pred))#, target_names=target_names))
# Y_test.head()

=====related================
              precision    recall  f1-score   support

           0       0.66      0.45      0.53      1608
           1       0.84      0.93      0.88      4946

    accuracy                           0.81      6554
   macro avg       0.75      0.69      0.71      6554
weighted avg       0.79      0.81      0.79      6554

=====request================
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      5459
           1       0.78      0.43      0.55      1095

    accuracy                           0.88      6554
   macro avg       0.84      0.70      0.74      6554
weighted avg       0.87      0.88      0.87      6554

=====offer================
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6530
           1       0.00      0.00      0.00        24

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50

c:\src\MLpipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\src\MLpipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\src\MLpipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\src\MLpipeline\venv\lib\site-packages\sklearn\metrics\_cla

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6133
           1       0.20      0.00      0.01       421

    accuracy                           0.93      6554
   macro avg       0.57      0.50      0.49      6554
weighted avg       0.89      0.93      0.90      6554

=====transport================
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6243
           1       0.71      0.14      0.24       311

    accuracy                           0.96      6554
   macro avg       0.84      0.57      0.61      6554
weighted avg       0.95      0.96      0.94      6554

=====buildings================
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6233
           1       0.75      0.14      0.24       321

    accuracy                           0.96      6554
   macro avg       0.86      0.57      0.61      6554
weighted avg

c:\src\MLpipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\src\MLpipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\src\MLpipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\src\MLpipeline\venv\lib\site-packages\sklearn\metrics\_cla

### 6. Improve your model
Use grid search to find better parameters. 

In [50]:
parameters = {
    'clf__estimator__n_estimators': [10, 20, 40],
}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [51]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
cv.fit(X_train, Y_train)
Y_pred2 = cv.predict(X_test)

[Pipeline] .............. (step 1 of 3) Processing vect, total=   3.5s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=  24.8s
[Pipeline] .............. (step 1 of 3) Processing vect, total=   3.4s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=  25.8s
[Pipeline] .............. (step 1 of 3) Processing vect, total=   3.9s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=  27.5s
[Pipeline] .............. (step 1 of 3) Processing vect, total=   3.9s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=  28.4s
[Pipeline] .............. (step 1 of 3) Processing vect, total=   4.0s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipel

In [54]:
for ix,col in enumerate(Y_train.columns):
    y_true = Y_test[col]
    y_pred = Y_pred2[:,ix]
    target_names = ['0', '1']
    print(f"====={col}================")
    print(classification_report(y_true, y_pred))#, target_names=target_names))

=====related================
              precision    recall  f1-score   support

           0       0.68      0.41      0.51      1552
           1       0.84      0.94      0.89      5002

    accuracy                           0.82      6554
   macro avg       0.76      0.68      0.70      6554
weighted avg       0.80      0.82      0.80      6554

=====request================
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5408
           1       0.82      0.47      0.60      1146

    accuracy                           0.89      6554
   macro avg       0.86      0.73      0.77      6554
weighted avg       0.88      0.89      0.88      6554

=====offer================
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50

c:\src\MLpipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\src\MLpipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\src\MLpipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6309
           1       0.76      0.08      0.14       245

    accuracy                           0.96      6554
   macro avg       0.86      0.54      0.56      6554
weighted avg       0.96      0.96      0.95      6554

=====death================
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6269
           1       0.83      0.18      0.30       285

    accuracy                           0.96      6554
   macro avg       0.89      0.59      0.64      6554
weighted avg       0.96      0.96      0.95      6554

=====other_aid================
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5690
           1       0.50      0.03      0.05       864

    accuracy                           0.87      6554
   macro avg       0.69      0.51      0.49      6554
weighted avg    

c:\src\MLpipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\src\MLpipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\src\MLpipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\src\MLpipeline\venv\lib\site-packages\sklearn\metrics\_cla

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [55]:
import pickle
with open('cv1_pkl', 'wb') as files:
    pickle.dump(cv, files)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [57]:
with open('cv1_pkl' , 'rb') as f:
    lr = pickle.load(f)


In [59]:
lr.predict(X[:5])

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)